In [0]:
import csv
from faker import Faker
import random as rnd
from pyspark.sql import SparkSession
from faker.config import AVAILABLE_LOCALES
assert 'pt_BR' in AVAILABLE_LOCALES
from datetime import datetime
from zoneinfo import ZoneInfo

agora_brasilia = datetime.now(ZoneInfo("America/Sao_Paulo"))

spark = SparkSession.Builder().appName("taskInputDataRandom").getOrCreate()

fake = Faker('pt_BR')
n_obs = rnd.randint(100, 50000)

def profile():
    return {
        'nome': fake.name(),
        'cpf': fake.cpf(),
        'telefone': fake.phone_number(),
        'dataNascimento': fake.date_of_birth(),
        'email': fake.email(),
        'endereco': fake.address().replace('\n', ' | '),
        'cargo': fake.job(),
        'empresa': fake.company(),
        'website': fake.url()
    }

dados_finais = []

dataHoraExecucao = datetime.now(ZoneInfo("America/Sao_Paulo"))

dataExecucao = dataHoraExecucao.strftime("%Y%m%d")
horaExecucao = dataHoraExecucao.strftime("%H%M%S")

for obs in range(n_obs):
    dados_finais.append(profile())                                                               

df = spark.createDataFrame(dados_finais)
df.display()

df.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .option("sep", ";") \
    .csv(f'/Volumes/dados_clientes/raw/arqs_recebidos/dados_clientes_{dataExecucao}_{horaExecucao}')